Hugging face maintains a central hub of datasets, can load popular datasets like IMDB

from datasets import load_dataset
dataset = load_dataset("imdb")

Access to evaluation metrics

from datasets import load_metric
metric = load_metric("accuracy")
print(metric.compute(preictions=[0,1,0], references = [0,1,1]))


works with pytorch
converts datasets into  torch.util.data.Dataset
tf.data.Dataset

In [11]:
from datasets import load_dataset

In [12]:
dataset = load_dataset("jamescalam/ai-arxiv2",split="train")
dataset

Dataset({
    features: ['id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'content', 'references'],
    num_rows: 2673
})

In [13]:
import os 
from getpass import getpass
from dotenv import load_dotenv
from semantic_router.encoders import OpenAIEncoder
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
encoder = OpenAIEncoder(name="text-embedding-3-small")

In [14]:
print(dataset["content"][1])

3 2 0 2
c e D 8 2 ] G L . s c [
1 v 8 3 2 7 1 . 2 1 3 2 : v i X r a
# Fast Inference of Mixture-of-Experts Language Models with Offloading
Artyom Eliseev Moscow Institute of Physics and Technology Yandex School of Data Analysis lavawolfiee@gmail.com
# Denis Mazur Moscow Institute of Physics and Technology Yandex Researchcore denismazur8@gmail.com
# Abstract
With the widespread adoption of Large Language Models (LLMs), many deep learning practitioners are looking for strategies of running these models more efficiently. One such strategy is to use sparse Mixture-of-Experts (MoE) â a type of model architectures where only a fraction of model layers are active for any given input. This property allows MoE-based language models to generate tokens faster than their âdenseâ counterparts, but it also increases model size due to having multiple âexpertsâ. Unfortunately, this makes state-of-the-art MoE language models difficult to run without high-end GPUs. In this work, we study the p

In [18]:
content = dataset[3]["content"]
print(content[:1000])

# Mamba: Linear-Time Sequence Modeling with Selective State Spaces
# Albert Gu*1 and Tri Dao*2
1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me
# Abstract
Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many subquadratic-time architectures such as linear attention, gated convolution and recurrent models, and structured state space models (SSMs) have been developed to address Transformersâ computational ineï¬ciency on long sequences, but they have not performed as well as attention on important modalities such as language. We identify that a key weakness of such models is their inability to perform content-based reasoning, and make several improvements. First, simply letting the SSM parameters be functions of the input addresses their weakness with discrete modalities

In [15]:
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
logger.setLevel("Warning")
splitter = RollingWindowSplitter(
    encoder = encoder,
    min_split_tokens = 100,
    max_split_tokens = 500,
    window_size=2,
    plot_splits = True, #set this to true to visualize chunking
    enable_statistics = True
)

ModuleNotFoundError: No module named 'semantic_router.splitters'

In [16]:
import semantic_router
print("Installed version:", semantic_router.__version__)
print("Installed at:", semantic_router.__file__)

Installed version: 0.1.2
Installed at: d:\agenticai\.venv\Lib\site-packages\semantic_router\__init__.py


In [19]:
from semantic_chunkers import StatisticalChunker
chunker = StatisticalChunker(encoder=encoder)   
chunks = chunker(docs=[content])

2025-08-31 20:01:26 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically merging.
100%|██████████| 49/49 [01:39<00:00,  2.04s/it]


In [21]:
chunker.print(chunks[0])

Split 1, tokens 300, triggered by: token limit
# Mamba: Linear-Time Sequence Modeling with Selective State Spaces # Albert Gu*1 and Tri Dao*2 1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me # Abstract Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many subquadratic-time architectures such as linear attention, gated convolution and recurrent models, and structured state space models (SSMs) have been developed to address Transformersâ computational ineï¬ ciency on long sequences, but they have not performed as well as attention on important modalities such as language. We identify that a key weakness of such models is their inability to perform content-based reasoning, and make several improvements. First, simply letting the SSM parameters be functions of the input addr

CONSECUTIVE CHUNKING

In [22]:
from semantic_chunkers import ConsecutiveChunker
chunker = ConsecutiveChunker(encoder=encoder, score_threshold=0.3)

In [23]:
chunks = chunker(docs=[content])

100%|██████████| 3086/3086 [00:00<00:00, 23013.58it/s]


In [24]:
chunker.print(chunks[0])

Split 1, tokens None, triggered by: 0.09
# Mamba:
----------------------------------------------------------------------------------------


Split 2, tokens None, triggered by: 0.10
Linear-Time Sequence Modeling with Selective State Spaces
----------------------------------------------------------------------------------------


Split 3, tokens None, triggered by: 0.25
# Albert Gu*1 and Tri Dao*2 1Machine Learning Department, Carnegie Mellon University 2Department of Computer Science, Princeton University agu@cs.cmu.edu, tri@tridao.me
----------------------------------------------------------------------------------------


Split 4, tokens None, triggered by: 0.22
# Abstract
----------------------------------------------------------------------------------------


Split 5, tokens None, triggered by: 0.30
Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many sub

CUMULATIVE SEMANTIC CHUNKING

In [ ]:
from semantic_chunkers import CumulativeChunker
chunker = CumulativeChunker(encoder=encoder, score_threshold=0.3)
chunks = chunker(docs=[content])